In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace


In [2]:
flags = Namespace(
    train_file='harry.txt',
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['I', 'am'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
)


def get_data_from_file(train_file, batch_size, seq_size):
    with open(train_file, 'r') as f:
        text = f.read()
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))
    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))
    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [3]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]


In [4]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)
        
    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))


def get_loss_and_train_op(net, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    return criterion, optimizer


In [13]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
        flags.train_file, flags.batch_size, flags.seq_size)

    net = RNNModule(n_vocab, flags.seq_size,
                    flags.embedding_size, flags.lstm_size)
    net = net.to(device)

    criterion, optimizer = get_loss_and_train_op(net, 0.01)

    iteration = 0
    epochs = 200
    for e in range(epochs):
        batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
        state_h, state_c = net.zero_state(flags.batch_size)
        
        # Transfer data to GPU
        state_h = state_h.to(device)
        state_c = state_c.to(device)
        for x, y in batches:
            iteration += 1
            
            # Tell it we are in training mode
            net.train()

            # Reset all gradients
            optimizer.zero_grad()

            # Transfer data to GPU
            x = torch.tensor(x).to(device)
            y = torch.tensor(y).to(device)

            logits, (state_h, state_c) = net(x, (state_h, state_c))
            loss = criterion(logits.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss_value = loss.item()

            # Perform back-propagation
            loss.backward()
            
            _ = torch.nn.utils.clip_grad_norm_(net.parameters(), flags.gradients_norm)

            # Update the network's parameters
            optimizer.step()
            
            if iteration % 100 == 0:
                print('Epoch: {}/{}'.format(e, epochs),
                      'Iteration: {}'.format(iteration),
                      'Loss: {}'.format(loss_value))

            if iteration % 1000 == 0:
                predict(device, net, flags.initial_words, n_vocab,
                        vocab_to_int, int_to_vocab, top_k=5)
                torch.save(net.state_dict(),
                           'model-{}.pth'.format(iteration))




In [6]:
def predict(device, net, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    net.eval()

    state_h, state_c = net.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)
    for w in words:
        ix = torch.tensor([[vocab_to_int[w]]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words.append(int_to_vocab[choice])
    
    for _ in range(100):
        ix = torch.tensor([[choice]]).to(device)
        output, (state_h, state_c) = net(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words.append(int_to_vocab[choice])

    print(' '.join(words))

True

In [14]:
if __name__ == '__main__':
    main()

Vocabulary size 11897
Epoch: 0/200 Iteration: 100 Loss: 7.282401084899902
Epoch: 1/200 Iteration: 200 Loss: 6.546903610229492
Epoch: 1/200 Iteration: 300 Loss: 6.130568981170654
Epoch: 2/200 Iteration: 400 Loss: 5.607007026672363
Epoch: 3/200 Iteration: 500 Loss: 5.276131629943848
Epoch: 3/200 Iteration: 600 Loss: 4.964814186096191
Epoch: 4/200 Iteration: 700 Loss: 4.607592582702637
Epoch: 5/200 Iteration: 800 Loss: 4.424565315246582
Epoch: 5/200 Iteration: 900 Loss: 4.121367454528809
Epoch: 6/200 Iteration: 1000 Loss: 4.16901969909668
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of his head, and they had been

Epoch: 20/200 Iteration: 3100 Loss: 2.473079204559326
Epoch: 20/200 Iteration: 3200 Loss: 2.3404061794281006
Epoch: 21/200 Iteration: 3300 Loss: 2.4034786224365234
Epoch: 22/200 Iteration: 3400 Loss: 2.2350354194641113
Epoch: 22/200 Iteration: 3500 Loss: 2.2232515811920166
Epoch: 23/200 Iteration: 3600 Loss: 2.2954514026641846
Epoch: 24/200 Iteration: 3700 Loss: 2.0879781246185303
Epoch: 24/200 Iteration: 3800 Loss: 2.1979031562805176
Epoch: 25/200 Iteration: 3900 Loss: 1.9427220821380615
Epoch: 26/200 Iteration: 4000 Loss: 2.149099826812744
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of his head, and they ha

Epoch: 33/200 Iteration: 5100 Loss: 1.6990004777908325
Epoch: 33/200 Iteration: 5200 Loss: 1.9624916315078735
Epoch: 34/200 Iteration: 5300 Loss: 1.8133317232131958
Epoch: 35/200 Iteration: 5400 Loss: 1.8436137437820435
Epoch: 35/200 Iteration: 5500 Loss: 1.608730673789978
Epoch: 36/200 Iteration: 5600 Loss: 1.7350512742996216
Epoch: 37/200 Iteration: 5700 Loss: 1.764054536819458
Epoch: 37/200 Iteration: 5800 Loss: 1.6773797273635864
Epoch: 38/200 Iteration: 5900 Loss: 1.5485005378723145
Epoch: 39/200 Iteration: 6000 Loss: 1.6353843212127686
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of his head, and they ha

Epoch: 46/200 Iteration: 7100 Loss: 1.4603289365768433
Epoch: 47/200 Iteration: 7200 Loss: 1.667672872543335
Epoch: 47/200 Iteration: 7300 Loss: 1.450778603553772
Epoch: 48/200 Iteration: 7400 Loss: 1.3853739500045776
Epoch: 49/200 Iteration: 7500 Loss: 1.5182061195373535
Epoch: 49/200 Iteration: 7600 Loss: 1.2786388397216797
Epoch: 50/200 Iteration: 7700 Loss: 1.2818673849105835
Epoch: 50/200 Iteration: 7800 Loss: 1.4282432794570923
Epoch: 51/200 Iteration: 7900 Loss: 1.4418888092041016
Epoch: 52/200 Iteration: 8000 Loss: 1.5171122550964355
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of his head, and they ha

Epoch: 59/200 Iteration: 9100 Loss: 1.3339773416519165
Epoch: 60/200 Iteration: 9200 Loss: 1.1915305852890015
Epoch: 60/200 Iteration: 9300 Loss: 1.2992178201675415
Epoch: 61/200 Iteration: 9400 Loss: 1.5309725999832153
Epoch: 62/200 Iteration: 9500 Loss: 1.3890589475631714
Epoch: 62/200 Iteration: 9600 Loss: 1.461835265159607
Epoch: 63/200 Iteration: 9700 Loss: 1.4834399223327637
Epoch: 64/200 Iteration: 9800 Loss: 1.3311622142791748
Epoch: 64/200 Iteration: 9900 Loss: 1.356078028678894
Epoch: 65/200 Iteration: 10000 Loss: 1.3664391040802002
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of his head, and they h

...

Epoch: 124/200 Iteration: 19100 Loss: 1.0717321634292603
Epoch: 125/200 Iteration: 19200 Loss: 1.06285560131073
Epoch: 126/200 Iteration: 19300 Loss: 1.1475056409835815
Epoch: 126/200 Iteration: 19400 Loss: 1.037143588066101
Epoch: 127/200 Iteration: 19500 Loss: 1.0238584280014038
Epoch: 128/200 Iteration: 19600 Loss: 1.1668808460235596
Epoch: 128/200 Iteration: 19700 Loss: 0.9044299125671387
Epoch: 129/200 Iteration: 19800 Loss: 1.2412054538726807
Epoch: 130/200 Iteration: 19900 Loss: 1.3299534320831299
Epoch: 130/200 Iteration: 20000 Loss: 1.1980282068252563
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of hi

Epoch: 131/200 Iteration: 20100 Loss: 1.163682222366333
Epoch: 132/200 Iteration: 20200 Loss: 1.2142210006713867
Epoch: 132/200 Iteration: 20300 Loss: 1.1637758016586304
Epoch: 133/200 Iteration: 20400 Loss: 1.0255763530731201
Epoch: 133/200 Iteration: 20500 Loss: 1.2212367057800293
Epoch: 134/200 Iteration: 20600 Loss: 1.037124752998352
Epoch: 135/200 Iteration: 20700 Loss: 1.143819808959961
Epoch: 135/200 Iteration: 20800 Loss: 0.9842146039009094
Epoch: 136/200 Iteration: 20900 Loss: 0.9604178071022034
Epoch: 137/200 Iteration: 21000 Loss: 1.1847115755081177
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of hi

Epoch: 137/200 Iteration: 21100 Loss: 1.2127383947372437
Epoch: 138/200 Iteration: 21200 Loss: 1.0505884885787964
Epoch: 139/200 Iteration: 21300 Loss: 1.1328468322753906
Epoch: 139/200 Iteration: 21400 Loss: 1.113013505935669
Epoch: 140/200 Iteration: 21500 Loss: 1.0729351043701172
Epoch: 141/200 Iteration: 21600 Loss: 0.9885857105255127
Epoch: 141/200 Iteration: 21700 Loss: 1.144831895828247
Epoch: 142/200 Iteration: 21800 Loss: 0.9739525318145752
Epoch: 143/200 Iteration: 21900 Loss: 1.0493757724761963
Epoch: 143/200 Iteration: 22000 Loss: 1.2205469608306885
I am learned of course, with a lot of course, he was going on? Harry and the back down on its hinges by a soft hooting and a very good said Ron and Ron and a lot like good reason, you know, there are dragons -- he's gone. It was very curious Finally that made all about it, he had never believed I don't know how many Galleons they could hear Harry picked the station. There is going out to his eyes on his wand in the only one of h

Epoch: 144/200 Iteration: 22100 Loss: 0.9827046990394592
Epoch: 145/200 Iteration: 22200 Loss: 1.211698055267334
Epoch: 145/200 Iteration: 22300 Loss: 1.2295033931732178
Epoch: 146/200 Iteration: 22400 Loss: 0.9125105142593384
Epoch: 147/200 Iteration: 22500 Loss: 1.128033995628357
Epoch: 147/200 Iteration: 22600 Loss: 1.0288463830947876
Epoch: 148/200 Iteration: 22700 Loss: 0.8811599612236023
Epoch: 149/200 Iteration: 22800 Loss: 1.0941559076309204
Epoch: 149/200 Iteration: 22900 Loss: 0.9603424668312073
Epoch: 150/200 Iteration: 23000 Loss: 0.7920719385147095


KeyboardInterrupt: 